In [3]:
import pandas as pd
import re
import string
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import pandas as pd
import numpy as np
import scipy as sp
import sklearn
import sys
import nltk
from nltk.corpus import stopwords
from gensim.models import ldamodel
import gensim.corpora
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.decomposition import NMF
from sklearn.preprocessing import normalize
import pickle
from gensim.models import LdaModel
from gensim.corpora import Dictionary
import pyLDAvis.gensim_models
import warnings
from itertools import chain
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()
import numpy as np 
import pandas as pd
import string
import re
from tqdm._tqdm_notebook import tqdm_notebook,tnrange,tqdm
from collections import Counter,OrderedDict
from gensim import models,corpora
from gensim.summarization import summarize,keywords
import warnings
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
import seaborn as sns
import pyLDAvis.gensim_models
import gensim.models.phrases as gen
from gensim.models.coherencemodel import CoherenceModel

ImportError: cannot import name 'has_pattern' from 'gensim.utils' (/Users/cheffbcookin/miniconda3/lib/python3.9/site-packages/gensim/utils.py)

In [14]:
!pip uninstall gensim

Found existing installation: gensim 4.1.2
Uninstalling gensim-4.1.2:
  Would remove:
    /Users/cheffbcookin/miniconda3/lib/python3.9/site-packages/gensim-4.1.2.dist-info/*
    /Users/cheffbcookin/miniconda3/lib/python3.9/site-packages/gensim/*
Proceed (Y/n)? ^C
ERROR: Operation cancelled by user


In [ ]:
!pip3 uninstall pandas

In [3]:
!pip install pandas

  Using cached pandas-1.4.2-cp39-cp39-macosx_10_9_x86_64.whl (11.1 MB)


In [2]:
import sys
print(sys.executable)

print(sys.path)

/Users/cheffbcookin/miniconda3/bin/python
['/Users/cheffbcookin/Desktop/Spark/NAACP-TopicModeling-Spring22', '/Users/cheffbcookin/miniconda3/lib/python39.zip', '/Users/cheffbcookin/miniconda3/lib/python3.9', '/Users/cheffbcookin/miniconda3/lib/python3.9/lib-dynload', '', '/Users/cheffbcookin/.local/lib/python3.9/site-packages', '/Users/cheffbcookin/miniconda3/lib/python3.9/site-packages']


In [ ]:
# Loading Data
Download Data from https://www.kaggle.com/hsankesara/medium-articles

In [2]:
!ls

00154dd0-3c43-4c79-929b-daf93c654489.csv
LDA_ex1.ipynb
Pre_processing.ipynb
README.md
practice.csv


In [ ]:
data=pd.read_csv('practice.csv')
len(data)

In [ ]:
data.head(5)

In [ ]:
data['length_text'] = data['text'].str.len()
sns.distplot(data['length_text'], color="r")
plt.show()

In [ ]:
data['length_title'] = data['title'].str.len()
sns.distplot(data['length_title'], color="r")
plt.show()

In [ ]:
titles = [x for x in data['title']]
docs = [x for x in data['text']]

In [ ]:
nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

# Data Preprocessing
We use NLTK’s Wordnet to find the meanings of words, synonyms, antonyms, and more. In addition, we use WordNetLemmatizer to get the root word. Filter out stop words.

In [ ]:
# A function to prepare the text for topic modelling
def words(text):
    regex = re.compile('[' + re.escape(string.punctuation) + '0-9\\r\\t\\n]')
    text = regex.sub(" ", text.lower())
    words = text.split(" ")
    words = [re.sub('\S*@\S*\s?', '', sent) for sent in words]
    words = [re.sub('\s+', ' ', sent) for sent in words]
    words = [re.sub("\'", "", sent) for sent in words]
    words = [w for w in words if not len(w) < 2]
    words = [w for w in words if w not in stop_words]
    words = [lmtzr.lemmatize(w) for w in words]
    
    return words

In [ ]:
nltk.download('wordnet')
docs = [words(x) for x in data['text']]

In [ ]:
 # Create a dictionary representation of the documents.
dictionary = Dictionary(docs)
print('Number of unique words in initital documents:', len(dictionary))

# Filter out words that occur less than 10 documents, or more than 20% of the documents.
dictionary.filter_extremes(no_below=10, no_above=0.2)
print('Number of unique words after removing rare and common words:', len(dictionary))

In [ ]:
corpus = [dictionary.doc2bow(doc) for doc in docs]
#print(len(corpus))
#corpus[336]
bow_doc_300 = corpus[300]

for i in range(len(bow_doc_300)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_300[i][0], 
                                                     dictionary[bow_doc_300[i][0]], 
                                                     bow_doc_300[i][1]))

# Topic Modeling using LDA
LDA :: Latent Dirichlet Allocation is a probabilistic model. It is a unsupervised machine learning technique. And to obtain cluster assignments, it uses two probability values: P( word | topics) and P( topics | documents).

pyLDAvis is designed to help users interpret the topics in a topic model that has been fit to a corpus of text data. The package extracts information from a fitted LDA topic model to inform an interactive web-based visualization.

The size of the bubble measures the importance of the topics, relative to the data.
When we have 5 or 10 topics, we can see certain topics are clustered together (overlapping bubbles), this indicates the similarity between topics.

In [ ]:
def get_lda_topics(model, num_topics):
    word_dict = {};
    for i in range(num_topics):
        words = model.show_topic(i, topn = 20);
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = [i[0] for i in words];
    return pd.DataFrame(word_dict);

In [ ]:
get_lda_topics(lda_model, 10)

In [ ]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=500,
                                           passes=20,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
pyLDAvis.enable_notebook()
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
pyLDAvis.gensim.prepare(model, corpus, dictionary)

The above is the visual representation of the topic modelling and the below is the raw representation and we can specify #words with highest probability to display in each topic. 

In [ ]:
lda_model.save('model10.gensim')
topics = lda_model.print_topics(num_words=6)
for topic in topics:
    print(topic)

In [ ]:
#Performance evaluation by classifying sample document using LDA Bag of Words model
for index, score in sorted(lda_model[corpus[300]], key=lambda tup: -1*tup[1]) :
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 40)))

# Topic Modeling using NMF

In [ ]:
data_text = data[['text']]
data_text = data_text.astype('str')

In [ ]:
articles = [value[0] for value in data_text.iloc[0:].values]

In [ ]:
articles_sentences = [' '.join(text) for text in articles]

In [ ]:
# obtain a Counts design matrix
vectorizer = CountVectorizer(analyzer='word', max_features=1000);
x_counts = vectorizer.fit_transform(articles_sentences);

In [ ]:
# set a TfIdf transformer, and transfer the counts with the model.
transformer = TfidfTransformer()
x_tfidf = transformer.fit_transform(x_counts);

In [ ]:
#normalize the TfIdf values
xtfidf_norm = normalize(x_tfidf, norm='l1', axis=1)

In [ ]:
num_topics = 10
model = NMF(n_components=num_topics, init='nndsvd');
model.fit(xtfidf_norm)

In [ ]:
## Get words from each topic 
def get_nmf_topics(model, n_top_words):
    #the word ids obtained need to be reverse-mapped to the words so we can print the topic names.
    feat_names = vectorizer.get_feature_names()
    word_dict = {};
    for i in range(num_topics):
        words_ids = model.components_[i].argsort()[:-20 - 1:-1]
        words = [feat_names[key] for key in words_ids]
        words = [re.sub('\S*@\S*\s?', '', sent) for sent in words]
        words = [re.sub('\s+', ' ', sent) for sent in words]
        words = [re.sub("\'", "", sent) for sent in words]
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = words;
    return pd.DataFrame(word_dict);

In [ ]:
get_nmf_topics(model, 20)

# Cosine Similarity b/w Documents
Given a keyword, Document Recommender system can suggest you the best documents from the pool of documents. 

In [ ]:
words_list=list(chain.from_iterable(docs))

In [ ]:
# Cosine similarity
def cosine_sim(text1, text2):
    tfidf_score = TfidfVectorizer().fit_transform([text1, text2])
    return ((tfidf_score * tfidf_score.T).A)[0, 1]

# Most similar article
def closest_doc_name(sentence, docs):
    cos = []
    for i in range(len(docs)):
        cos.append(cosine_sim(', '.join(sentence.split(' ')),', '.join(docs[i])))
    return [titles[x] for x in np.argsort(cos)[-10:][::-1]]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(tokenizer=words, stop_words=stop_words).fit(words_list)

In [ ]:
closest_doc_name('news', docs)

# References
https://towardsdatascience.com/topic-modelling-in-python-with-nltk-and-gensim-4ef03213cd21

https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24

https://medium.com/ml2vec/topic-modeling-is-an-unsupervised-learning-approach-to-clustering-documents-to-discover-topics-fdfbf30e27df